In [1]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from keras.preprocessing.image import *
from tensorflow.keras.preprocessing import *

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-04-22 03:22:08.661194: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 03:22:08.763491: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-22 03:22:08.788569: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory:

In [2]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=100)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=100)

In [3]:
inputs = Input(shape=(32,32, 3))

# Layer 1
x = Conv2D(32, (3,3), activation='relu', kernel_regularizer=l2(1e-6))(inputs)

# Layer 2
x = Conv2D(32, (3,3), activation='relu', kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 3
x = Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(1e-6))(x)

# Layer 4
x = Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 5
x = AveragePooling2D()(x)
x = Flatten()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(1e-6))(x)

# Layer 6
outputs = Dense(100, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                             

2023-04-22 03:22:11.019440: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 03:22:11.655028: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22277 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6


In [4]:
# 옵티마이저 및 학습률 스케줄러 정의
# 옵티마이저 정의
learning_rate = 1e-3

def learning_rate_schedule(epoch):
    new_learning_rate = learning_rate

    if epoch <= 50:
        pass
    elif epoch > 50 and epoch <= 100:
        new_learning_rate = learning_rate * 0.1
    else:
        new_learning_rate = learning_rate * 0.01
        
    print('Learning rate:', new_learning_rate)
    
    return new_learning_rate


opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate )


model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [5]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip= True
                             )

# 이미지 데이터의 분포를 조정
datagen.fit(x_train)

In [6]:
from tensorflow.keras import callbacks

wandb.init(project="CIFAR100", entity="hcim", name='LeNet-relu')

wandbCallback=wandb.keras.WandbCallback(monitor="val_accuracy", mode="max", 
                                        log_weights=True, log_gradients=True, 
                                        training_data=datagen.flow(x_train, y_train, batch_size=128))


callbacks = [wandbCallback, LearningRateScheduler(learning_rate_schedule)]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chan4im (hcim). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [7]:
model.fit(datagen.flow(x_train, y_train, batch_size=128),
          validation_data=(x_test, y_test),
          epochs=200, workers=4,
          callbacks=callbacks)

Learning rate: 0.001
Epoch 1/200


2023-04-22 03:22:17.086094: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-04-22 03:22:18.385742: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


391/391 [==============================] - ETA: 0s - loss: 4.1744 - accuracy: 0.0627

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_032214-41jqmhl5/files/model-best/assets


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_032214-41jqmhl5/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230422_032214-41jqmhl5/files/model-best)... Done. 0.0s


...

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_032214-41jqmhl5/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230422_032214-41jqmhl5/files/model-best)... Done. 0.0s


391/391 [==============================] - 22s 55ms/step - loss: 1.4822 - accuracy: 0.5886 - val_loss: 2.0814 - val_accuracy: 0.4794 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 93/200
391/391 [==============================] - 21s 53ms/step - loss: 1.4794 - accuracy: 0.5853 - val_loss: 2.0870 - val_accuracy: 0.4735 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 94/200
391/391 [==============================] - 20s 50ms/step - loss: 1.4772 - accuracy: 0.5886 - val_loss: 2.0846 - val_accuracy: 0.4690 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 95/200
391/391 [==============================] - 20s 51ms/step - loss: 1.4726 - accuracy: 0.5888 - val_loss: 2.0772 - val_accuracy: 0.4746 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 96/200
391/391 [==============================] - 20s 51ms/step - loss: 1.4707 - accuracy: 0.5907 - val_loss: 2.0839 - val_accuracy: 0.4731 - lr: 1.0000e-04
Learning rate: 1e-05
Epoch 102/200
391/391 [==============================] - 20s 51ms/step - loss: 1.4540 - a

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_032214-41jqmhl5/files/model-best/assets


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_032214-41jqmhl5/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230422_032214-41jqmhl5/files/model-best)... Done. 0.0s


391/391 [==============================] - 22s 57ms/step - loss: 1.4304 - accuracy: 0.6014 - val_loss: 2.0796 - val_accuracy: 0.4795 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 157/200
391/391 [==============================] - 20s 52ms/step - loss: 1.4317 - accuracy: 0.6014 - val_loss: 2.0811 - val_accuracy: 0.4781 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 158/200
391/391 [==============================] - 20s 52ms/step - loss: 1.4330 - accuracy: 0.5988 - val_loss: 2.0780 - val_accuracy: 0.4789 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 159/200
391/391 [==============================] - 21s 53ms/step - loss: 1.4286 - accuracy: 0.6027 - val_loss: 2.0805 - val_accuracy: 0.4780 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 160/200
391/391 [==============================] - 20s 51ms/step - loss: 1.4289 - accuracy: 0.6039 - val_loss: 2.0826 - val_accuracy: 0.4783 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 161/200
391/391 [==============================] - 20s 52ms/step - loss: 1.4313 - a